In [ ]:
import pyscf.pbc.tools.pyscf_ase as pyscf_ase
import pyscf.pbc.gto as pbcgto
import pyscf.pbc.dft as pbcdft

import matplotlib.pyplot as plt
from ase.build import graphene
from ase.dft.kpoints import sc_special_points as special_points, get_bandpath

# Build Graphene Structure
a = 2.46  # Lattice constant in Å
graphene_cell = graphene(formula="C2", a=a, size=(1, 1, 1), vacuum=10)
print("Graphene volume:", graphene_cell.get_volume())

# Convert to PySCF Cell
cell = pbcgto.Cell()
cell.atom = pyscf_ase.ase_atoms_to_pyscf(graphene_cell)
cell.a = graphene_cell.cell  # Lattice vectors
cell.basis = 'gth-szv'
cell.pseudo = 'gth-pade'
cell.verbose = 5
cell.build(None, None)

# Define Band Path
points = special_points['hexagonal']
G = points['G']
M = points['M']
K = points['K']
band_kpts, kpath, sp_points = get_bandpath([G, M, K, G], graphene_cell.cell, npoints=100)
band_kpts = cell.get_abs_kpts(band_kpts)

# Perform Band Structure Calculation (Gamma Point Sampling)
mf = pbcdft.RKS(cell)
mf.kernel()

e_kn = mf.get_bands(band_kpts)[0]  # Band energies
vbmax = max(en[cell.nelectron // 2 - 1] for en in e_kn)  # Valence Band Maximum (VBM)
e_kn = [en - vbmax for en in e_kn]  # Align VBM to 0 eV

# Periodic Sampling (K-Point Grid)
kmf = pbcdft.KRKS(cell, cell.make_kpts([4, 4, 1]))  # 4x4x1 grid for graphene
mf.xc = 'b3lyp'
kmf.kernel()

e_kn_2 = kmf.get_bands(band_kpts)[0]  # Band energies
vbmax = max(en[cell.nelectron // 2 - 1] for en in e_kn_2)  # Valence Band Maximum (VBM)
e_kn_2 = [en - vbmax for en in e_kn_2]  # Align VBM to 0 eV

# Plot the Band Structure
au2ev = 27.21139  # Hartree to eV conversion
emin, emax = -5, 5  # Energy range in eV

plt.figure(figsize=(5, 6))
nbands = cell.nao_nr()
for n in range(nbands):
    plt.plot(kpath, [e[n] * au2ev for e in e_kn], color='#87CEEB', label="Γ-point Sampling" if n == 0 else "")
    plt.plot(kpath, [e[n] * au2ev for e in e_kn_2], color='#4169E1', label="4x4x1 K-Point Sampling" if n == 0 else "")
for p in sp_points:
    plt.axvline(p, color='k', linestyle='--', linewidth=0.5)
plt.axhline(0, color='k', linestyle='-', linewidth=0.5)
plt.xticks(sp_points, ['$%s$' % n for n in ['Γ', 'M', 'K', 'Γ']])
plt.axis(xmin=0, xmax=sp_points[-1], ymin=emin, ymax=emax)
plt.xlabel('Wave Vector')
plt.ylabel('Energy (eV)')
plt.title('Band Structure of Graphene')
plt.legend()
plt.show()